In [1]:
# Flask
from flask import Flask, make_response, request
app = Flask(__name__)

# Numpy
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import base64
from io import BytesIO

# Keras and Tensorflow
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
from keras.models import load_model

# Image resize functions
from skimage.transform import resize


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/

In [ ]:
# Initialize some dummy variables in case loading fails
img = plt.imread('/media/share/data/Tremella_fuciformis/T8-171027/image_N1.jpg')

# Load keras model
model = load_model('/media/share/jiaxin_cmu/Tremella_fuciformis_v1/Tremella_fuciformis.hdf5')

### Front page, just an interface for uploading a dicom ###
@app.route("/")
def index():
    return """
    <!DOCTYPE html>
    <html>
    <head>
    <title>Upload</title>
     <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js"></script>
     <link rel="stylesheet" media="screen" href ="static/bootstrap.min.css">
     <link rel="stylesheet" href="static/bootstrap-theme.min.css">
     <meta name="viewport" content = "width=device-width, initial-scale=1.0">
    </head>
    <body>
    <form id="upload-form" action="upload" method="POST" enctype="multipart/form-data">
        <label for="file">Please upload a Breast US dicom file (.dcm or .dicom):</label>
        <input id="file-picker" type="file" name="file" accept="*/*" multiple>
        <div id="msg" class="alert alert-info"></div>
        <input type="submit" value="Upload!" id="upload-button">
    </form>
    </body>
    <script>
        $("#file-picker").change(function(){
            var input = document.getElementById('file-picker');
            for (var i=0; i<input.files.length; i++)
            {
                var ext= input.files[i].name.substring(input.files[i].name.lastIndexOf('.')+1).toLowerCase()
                if ((ext == 'dcm') || (ext == 'dicom'))
                {
                    $("#msg").text("Files are supported")
                }
                else
                {
                    $("#msg").text("Files are NOT supported")
                    document.getElementById("file-picker").value ="";
                }
            }
        } );
    </script>
    </html>
    """

### Process uploaded dicom image ###
@app.route("/upload", methods=["POST"])
def upload():
    f = request.files['file']
    f.save('/media/share/data/Tremella_fuciformis/flask/tmp/tmp.jpg')

    # Read Image
    img = plt.imread('/media/share/data/Tremella_fuciformis/flask/tmp/tmp.jpg')
    bimg = BytesIO()
    
    # Plot with matplotlib, encode to PNG
    plt.figure(figsize=(15,10))
    plt.imshow(img)
    plt.savefig(bimg, format='png')
    bimg.seek(0)
    plot_encoded = base64.b64encode(bimg.getvalue()).decode()

    # Resize image
    if img.shape[0] > img.shape[1]:
        img = np.transpose(img, (1, 0, 2))
    
    img = resize(img, (256, 256, 3), mode='reflect')
    img = img[np.newaxis, :, :, :]
    
    output = model.predict(img)

    predicted = ('Above 60g'+str(1-output[0]) if output[0] < 0.5 else 'Below 60g'+str(output[0]))

    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Result</title>
    </head>
    <body>
    <h3> Prediction: {} </h3>
    <img src="data:image/png;base64,{}">
    </body>
    </html>
    """.format(predicted, plot_encoded)

app.run(host='0.0.0.0', port=8888)